In [39]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm
from tqdm import tqdm

from Chempy.parameter import ModelParameters

import sbi.utils as utils

import torch
from torch.distributions.normal import Normal
from torch.distributions.uniform import Uniform

import time as t
import pickle

import multiprocessing as mp

In [40]:
name = "NPE_C"

In [41]:
# ----- Load the data -----
a = ModelParameters()
labels = [a.to_optimize[i] for i in range(len(a.to_optimize))] + ['time']
priors = torch.tensor([[a.priors[opt][0], a.priors[opt][1]] for opt in a.to_optimize])

elements = a.elements_to_trace

In [42]:
import math
from typing import Callable, Optional

import torch

from sbi.sbi_types import Shape

class DispersedPosterior:
    def __init__(self, posterior: Callable, dispersion: float = 1.05):
        """give me a posterior and I'll disperse it. All calls to `sample` are wrapped.
        This class exploits: Var(aX) = a**2 * Var(X) for any random variable X
        while retaining the expectation value E[X] of all samples.

        Parameters:
            posterior: posterior distribution modelled like NeuralPosterior
            dispersion: choose values <1. to make the variance smaller,
                choose values >1. to make the variance larger (distribution more wide)
        """

        self.dispersion = math.sqrt(dispersion)
        self.posterior = posterior

    def set_default_x(self, x):
        self.posterior.set_default_x(x)

    def sample(
        self,
        sample_shape: Shape,
        x: Optional[torch.Tensor] = None,
        show_progress_bars: bool = False,
    ):
        value = self.posterior.sample(
            sample_shape, x, show_progress_bars=show_progress_bars
        )

        # obtain the median of all samples before applying
        # the dispersion to them (use median for more robust estimate)
        median = torch.median(value, dim=0)  # dim 0 is the batch dimension

        # disperse the samples
        dispersed = value * self.dispersion

        # obtain the new median after the dispersion
        median_ = torch.median(dispersed, dim=0)

        # shift to obtain the original expectation values
        # (we only want to disperse the samples, not offset)
        shift = median.values - median_.values

        return dispersed + shift

    def sample_batched(
        self,
        sample_shape: Shape,
        x: Optional[torch.Tensor] = None,
        show_progress_bars: bool = False,
    ):
        value = self.posterior.sample_batched(
            sample_shape, x, show_progress_bars=show_progress_bars
        )

        # obtain the median of all samples before applying
        # the dispersion to them (use median for more robust estimate)
        median = torch.median(value, dim=0)

        # disperse the samples
        dispersed = value * self.dispersion

        # obtain the new median after the dispersion
        median_ = torch.median(dispersed, dim=0)

        # shift to obtain the original expectation values
        # (we only want to disperse the samples, not offset)
        shift = median.values - median_.values

        return dispersed + shift

In [43]:
# ----- Load posterior -----
with open(f'data/posterior_{name}.pickle', 'rb') as f:
    posterior = pickle.load(f)

In [44]:
# ----- Load the NN -----
class Model_Torch(torch.nn.Module):
    def __init__(self):
        super(Model_Torch, self).__init__()
        self.l1 = torch.nn.Linear(len(labels), 100)
        self.l2 = torch.nn.Linear(100, 40)
        self.l3 = torch.nn.Linear(40, len(elements))

    def forward(self, x):
        x = torch.tanh(self.l1(x))
        x = torch.tanh(self.l2(x))
        x = self.l3(x)
        return x
    
model = Model_Torch()
model.load_state_dict(torch.load('data/pytorch_state_dict.pt'))
model.eval();

/tmp/ipykernel_1403316/3901098474.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('data/pytorch_state_dict.pt'))


# Setup priors
The inputs to the model are drawn from a fixed set of global galaxy parameters (slope of the IMF & Rate of Type Ia Supernove) and a set of local star parameters.

In [45]:
local_GP = utils.MultipleIndependent(
    [Normal(p[0]*torch.ones(1), p[1]*torch.ones(1)) for p in priors[2:]] +
    [Uniform(torch.tensor([2.0]), torch.tensor([12.8]))],
    validate_args=False)

global_GP = utils.MultipleIndependent(
    [Normal(p[0]*torch.ones(1), p[1]*torch.ones(1)) for p in priors[:2]],
    validate_args=False)

# Simulate data
With the set global priors and for each $N_{\rm stars}$ a set of local priors, we can simulate the data with the Neural Network.

In [46]:
N_stars = 1000
simulations = 1000

stars = local_GP.sample((N_stars,))
global_params = torch.tensor([[-2.3, -2.89]])

stars = torch.cat((global_params.repeat(N_stars, 1), stars), dim=1)

In [47]:
# ----- Simulate abundances -----
start = t.time()
abundances = model(stars)  
# Remove H from data, because it is just used for normalization (output with index 2)
abundances = torch.cat([abundances[:,0:2], abundances[:,3:]], axis=1)
end = t.time()
print(f'Time to create data for {N_stars} stars: {end-start:.3f} s')

Time to create data for 1000 stars: 0.041 s


# SBI with 5% observation error noise
Need to load the posterior trained with a 5% observation error

In [48]:
def add_noise(true_abundances):
    # Define observational erorrs
    pc_ab = 5 # percentage error in abundance

    # Jitter true abundances and birth-times by these errors to create mock observational values.
    obs_ab_errors = np.ones_like(true_abundances)*float(pc_ab)/100.
    obs_abundances = norm.rvs(loc=true_abundances,scale=obs_ab_errors)

    return obs_abundances

In [49]:
alpha_IMF_obs = []
log10_N_Ia_obs = []
simulations = 1000

start = t.time()
for i in tqdm(range(len(abundances))):
    x = add_noise(abundances[i].detach().numpy())
    alpha, N_Ia = posterior.sample((simulations,), x=x, show_progress_bars=False)[:,0:2].T
    alpha_IMF_obs.append(alpha)
    log10_N_Ia_obs.append(N_Ia)
end = t.time()
print(f'Time to run {simulations} simulations for {N_stars} stars: {end-start:.3f} s')

alpha_IMF_obs = np.array(alpha_IMF_obs)
log10_N_Ia_obs = np.array(log10_N_Ia_obs)

100%|██████████| 1000/1000 [00:12<00:00, 79.92it/s]

Time to run 1000 simulations for 1000 stars: 12.523 s


In [50]:
# save the data
np.save(f'data/alpha_IMF_obs_{name}.npy', alpha_IMF_obs)
np.save(f'data/log10_N_Ia_obs_{name}.npy', log10_N_Ia_obs)